In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime
import time
import numpy as np
import glob
import re
import pandas as pd

from FDIC import wsio, Logger
import FDIC.constants as paths

In [ ]:
path = '/Users/kris/data_sources/fdic/BulkReports/'

In [ ]:
for folder in glob.glob(path + '*[!.]'):
    print(f'{folder}')

In [ ]:
folders = glob.glob(path + '*[!.]')
print(f'{folders}')

In [ ]:
import pandas as pd
import datetime

df = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])
df0 = pd.DataFrame(columns=['Date', 'As_of'])
df1 = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])

#df = pd.concat([df, df1], ignore_index=True)

In [ ]:
df0 = pd.concat([df0, df1], ignore_index=True)

In [ ]:
df = pd.concat(df1, df1)

In [ ]:
#type(df['Date'])
#df['Date'].isin(['20240506'])

'20240506' in df['Date'].values

In [ ]:
df.shape[0]

In [ ]:
skipped_dtypes = {'Date': str, 'As_of': datetime.datetime}
skipped_dtypes.keys()

In [ ]:
import time

t = time.time()
period = 60

diff = t - period

current_time = time.strftime('%H:%M:%S', time.localtime(t))
wait_time = time.strftime('%H:%M:%S', time.localtime(diff))

milliseconds_current = int((t - int(t)) * 1000)
milliseconds_diff = int((diff - int(diff)) * 1000)

print(f'current: {current_time}.{milliseconds_current}\nwait: {wait_time}.{milliseconds_diff}\n')


In [ ]:
import time
import datetime

current = time.time()
#print(datetime.datetime.fromtimestamp(current + 3600).strftime("%Y%m%d %H:%M:%S"))
print(f'{time.strftime("%Y%m%d %H:%M:%S",time.gmtime(current + 3600))}')

In [ ]:
# Paths to your XBRL file and schema file

bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_base_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 09302023_Form051'
#schema_base_path = '/Users/kris/Downloads/_ 09302023_Form051/'

xbrl_file_path = bulkReport_path + 'ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
schema_file_path = schema_base_path + 'call-report051-2023-09-30-v261.xsd'
presentation_file_path = schema_base_path + 'call-report051-2023-09-30-v261-pres.xml'

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = '/Users/kris/data_sources/fdic/BulkReports/ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL'
#schema_file_path = '/Users/kris/Downloads/_ 09302023_Form051/call-report051-2023-09-30-v261.xsd'

def parse_xsd(schema_path):
    with open(schema_path, 'rb') as schema_file:
        schema_root = etree.XML(schema_file.read())
    return schema_root

def parse_xbrl(xbrl_path):
    with open(xbrl_path, 'rb') as xbrl_file:
        xbrl_root = etree.XML(xbrl_file.read())
    return xbrl_root

def extract_data_from_xbrl(xbrl_root, schema_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path):
    schema_root = parse_xsd(schema_file_path)
    xbrl_root = parse_xbrl(xbrl_file_path)
    data = extract_data_from_xbrl(xbrl_root, schema_root)
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def present_data(data):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Optionally, parse and use the presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    
    # Present data
    present_data(data)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    arcrole = 'http://www.xbrl.org/2003/arcrole/parent-child'
    role_refs = presentation_root.xpath('//link:roleRef', namespaces=link_namespace)
    
    presentations = {}
    for role_ref in role_refs:
        role_uri = role_ref.get('roleURI')
        locators = presentation_root.xpath(f'//link:loc[@xlink:role="{role_uri}"]', namespaces=link_namespace)
        arcs = presentation_root.xpath(f'//link:definitionArc[@xlink:arcrole="{arcrole}"]', namespaces=link_namespace)
        
        elements = []
        for arc in arcs:
            from_locator = arc.get('from')
            to_locator = arc.get('to')
            from_element = next((loc for loc in locators if loc.get('label') == from_locator), None)
            to_element = next((loc for loc in locators if loc.get('label') == to_locator), None)
            if from_element is not None and to_element is not None:
                elements.append((from_element.get('{http://www.w3.org/1999/xlink}href'), to_element.get('{http://www.w3.org/1999/xlink}href')))
        presentations[role_uri] = elements
    
    return presentations

def present_data(data, presentations):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    presentation_order = []
    for role_uri, elements in presentations.items():
        for parent, child in elements:
            presentation_order.append(child.split('#')[-1])
    
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentations = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentations)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "path/to/_09302023_Form051/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
    
    return presentation_order

def present_data(data, presentation_order):
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file and schema file
#xbrl_file_path = "/path/to/ABINGTON BANK_61476_20230930.XBRL"
#schema_file_path = "/path/to/_09302023_Form051/call-report051-2023-09-30-v261.xsd"
#presentation_file_path = "/mnt/data/call-report051-2023-09-30-v261-pres.xml"

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    with open(file_path, 'rb') as file:
        root = etree.XML(file.read())
    return root

def validate_xbrl(xbrl_root, schema_root):
    print("Validating XBRL file")
    schema = etree.XMLSchema(schema_root)
    if not schema.validate(xbrl_root):
        raise ValueError("XBRL file is not valid.")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_presentation_linkbase(presentation_root):
    print("Parsing presentation linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    xlink_namespace = {'xlink': 'http://www.w3.org/1999/xlink'}
    
    locators = {}
    for loc in presentation_root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    presentation_order = []
    hierarchy = {}
    for arc in presentation_root.xpath('//link:presentationArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            presentation_order.append(locators[to_label])
            if locators[from_label] not in hierarchy:
                hierarchy[locators[from_label]] = []
            hierarchy[locators[from_label]].append(locators[to_label])
    
    return presentation_order, hierarchy

def present_data(data, presentation_order, hierarchy):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print(df)

def main(xbrl_file_path, schema_file_path, presentation_file_path):
    print("Starting main process")
    schema_root = parse_xml(schema_file_path)
    xbrl_root = parse_xml(xbrl_file_path)
    #validate_xbrl(xbrl_root, schema_root)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse presentation linkbase
    presentation_root = parse_xml(presentation_file_path)
    presentation_order, hierarchy = parse_presentation_linkbase(presentation_root)
    
    # Present data
    present_data(data, presentation_order, hierarchy)

if __name__ == "__main__":
    main(xbrl_file_path, schema_file_path, presentation_file_path)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20231231.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    return calculations

# Function to parse the definition linkbase with additional debug statements
def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    try:
        root = etree.parse(definition_file)
    except Exception as e:
        print(f"Failed to parse definition linkbase file: {definition_file}. Error: {e}")
        return {}
    
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions
    
def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations, definitions):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display calculation relationships
    print("\nCalculations:")
    for parent, children in calculations.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    # Display definition relationships
    print("\nDefinitions:")
    for parent, children in definitions.items():
        print(f"{parent}:")
        for child in children:
            print(f"  - {child}")
    
    print("\nData:")
    print(df)

def main(xbrl_file_path, schema_files, linkbase_files):
    print("Starting main process")
    schema_roots = [parse_xml(file_path) for file_path in schema_files]
    xbrl_root = parse_xml(xbrl_file_path)
    
    #validate_xbrl(xbrl_root, schema_roots)
    
    # Extract data from XBRL instance
    data = extract_data_from_xbrl(xbrl_root)
    
    # Parse linkbases
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    definitions = parse_definition_linkbase(linkbase_files['definition'])
    
    # Parse additional linkbases if needed
    parse_additional_linkbases(linkbase_files['ec-mess'])
    parse_additional_linkbases(linkbase_files['ec'])
    
    # Present data
    present_data(data, presentation_order, hierarchy, calculations, definitions)

if __name__ == "__main__":
    main(xbrl_file_path, schema_files, linkbase_files)

In [ ]:
import os
from lxml import etree
import pandas as pd

# Paths to your XBRL file, schema file, and taxonomy files
bulkReport_path = '/Users/kris/data_sources/fdic/BulkReports/'
schema_path = '/Users/kris/Desktop/BankRegScraper/taxonomy/Call_ 12312023_Form051/'

xbrl_file_path = os.path.join(bulkReport_path, "ABINGTON BANK_61476/ABINGTON BANK_61476_20230930.XBRL")
schema_files = [
    os.path.join(schema_path, "call-report051-2023-12-31-v265.xsd"),
    os.path.join(schema_path, "concepts.xsd"),
    os.path.join(schema_path, "ffiec-instance-2004-06-10.xsd"),
    os.path.join(schema_path, "ffiec-linkbase-2004-05-14.xsd"),
    os.path.join(schema_path, "mdr_schema.xsd")
]
linkbase_files = {
    "presentation": os.path.join(schema_path, "call-report051-2023-12-31-v265-pres.xml"),
    "calculation": os.path.join(schema_path, "call-report051-2023-12-31-v265-calc.xml"),
    "definition": os.path.join(schema_path, "call-report051-2023-12-31-v265-def.xml"),
    "reference": os.path.join(schema_path, "call-report051-2023-12-31-v265-ref.xml"),
    "label": os.path.join(schema_path, "call-report051-2023-12-31-v265-instr.xml"),
    "ec-mess": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec-mess.xml"),
    "ec": os.path.join(schema_path, "call-report051-2023-12-31-v265-ec.xml")
}

def parse_xml(file_path):
    print(f"Parsing XML file: {file_path}")
    try:
        with open(file_path, 'rb') as file:
            root = etree.XML(file.read())
        return root
    except Exception as e:
        print(f"Failed to parse XML file: {file_path}. Error: {e}")
        raise

def validate_xbrl(xbrl_root, schema_roots):
    print("Validating XBRL file")
    for schema_root in schema_roots:
        print(f"Validating against schema: {schema_root}")
        schema = etree.XMLSchema(schema_root)
        if not schema.validate(xbrl_root):
            print(f"Validation failed against schema: {schema_root}")
            raise ValueError("XBRL file is not valid against the schema.")
        else:
            print(f"Validation succeeded against schema: {schema_root}")
    print("Validation completed")
    return True

def extract_data_from_xbrl(xbrl_root):
    print("Extracting data from XBRL")
    namespaces = {'xbrli': 'http://www.xbrl.org/2003/instance'}
    context_elements = xbrl_root.xpath('//xbrli:context', namespaces=namespaces)
    
    data = []
    for context in context_elements:
        context_id = context.get('id')
        data_elements = xbrl_root.xpath(f'//*[@contextRef="{context_id}"]', namespaces=namespaces)
        
        for element in data_elements:
            tag = etree.QName(element).localname
            value = element.text
            unit = element.get('unitRef', 'N/A')
            data.append((context_id, tag, value, unit))
    
    return data

def parse_linkbase(linkbase_file, arc_tag):
    print(f"Parsing {arc_tag} linkbase: {linkbase_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(linkbase_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    arcs = []
    for arc in root.xpath(f'//link:{arc_tag}Arc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        #print(f"Processing {arc_tag}Arc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Arcs found in {arc_tag} linkbase: {arcs}")
    return locators, arcs

def parse_presentation_linkbase(presentation_file):
    locators, presentation_arcs = parse_linkbase(presentation_file, 'presentation')
    presentation_order = []
    hierarchy = {}
    for parent, child in presentation_arcs:
        presentation_order.append(child)
        if parent not in hierarchy:
            hierarchy[parent] = []
        hierarchy[parent].append(child)
    
    return presentation_order, hierarchy

def parse_calculation_linkbase(calculation_file):
    print(f"Parsing calculation linkbase: {calculation_file}")
    locators, calculation_arcs = parse_linkbase(calculation_file, 'calculation')
    calculations = {}
    for parent, child in calculation_arcs:
        if parent not in calculations:
            calculations[parent] = []
        calculations[parent].append(child)
    
    print(f"Calculations: {calculations}")
    return calculations

def parse_definition_linkbase(definition_file):
    print(f"Parsing definition linkbase: {definition_file}")
    link_namespace = {'link': 'http://www.xbrl.org/2003/linkbase'}
    
    root = parse_xml(definition_file)
    locators = {}
    for loc in root.xpath('//link:loc', namespaces=link_namespace):
        label = loc.get('{http://www.w3.org/1999/xlink}label')
        href = loc.get('{http://www.w3.org/1999/xlink}href')
        locators[label] = href.split('#')[-1]
    
    print(f"Locators found: {locators}")
    
    definition_arcs = []
    for arc in root.xpath('//link:defArc', namespaces=link_namespace):
        from_label = arc.get('from')
        to_label = arc.get('to')
        print(f"Processing defArc from {from_label} to {to_label}")
        if from_label in locators and to_label in locators:
            definition_arcs.append((locators[from_label], locators[to_label]))
    
    print(f"Definition arcs found: {definition_arcs}")
    
    definitions = {}
    for parent, child in definition_arcs:
        if parent not in definitions:
            definitions[parent] = []
        definitions[parent].append(child)
    
    print(f"Definitions: {definitions}")
    return definitions

def parse_additional_linkbases(additional_file):
    # Add logic to parse additional linkbases if needed
    pass

def present_data(data, presentation_order, hierarchy, calculations=None, definitions=None):
    print("Presenting data")
    df = pd.DataFrame(data, columns=['Context ID', 'Tag', 'Value', 'Unit'])
    
    # Use presentation order to categorize and sort tags
    df['Tag'] = pd.Categorical(df['Tag'], categories=presentation_order, ordered=True)
    df.sort_values('Tag', inplace=True)
    
    # Display the hierarchical structure
    print("Hierarchy:")
    for parent, children in hierarchy.items():
        print(f"{parent}: {children}")
    
    # Display calculations
    if calculations:
        print("\nCalculations:")
        for parent, children in calculations.items():
            print(f"{parent}: {children}")
    
    # Display definitions
    if definitions:
        print("\nDefinitions:")
        for parent, children in definitions.items():
            print(f"{parent}: {children}")

def main():
    # Parsing the provided definition linkbase
    definitions = parse_definition_linkbase(linkbase_files['definition'])

    # Parsing other necessary linkbases
    calculations = parse_calculation_linkbase(linkbase_files['calculation'])
    presentation_order, hierarchy = parse_presentation_linkbase(linkbase_files['presentation'])

    # Parsing and validating XBRL data
    xbrl_root = parse_xml(xbrl_file_path)
    schema_roots = [parse_xml(schema_file) for schema_file in schema_files]
    #validate_xbrl(xbrl_root, schema_roots)

    # Extracting data from XBRL
    data = extract_data_from_xbrl(xbrl_root)

    # Presenting data
    present_data(data, presentation_order, hierarchy, calculations=calculations, definitions=definitions)

if __name__ == "__main__":
    main()

In [ ]:
import xml.etree.ElementTree as ET
import FDIC.wsio as wsio


filepath = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'

def ParseXBRL(self, filepath):
    """
    Parses Call Reports to extract all MDRM item and value from XBRL file
    """
    # Parse XML
    tree  = ET.parse(filepath)
    root = tree.getroot()

    report_values = []
    for child in root:
        #print(child.tag, child.attrib)
        code = child.tag[child.tag.find('}')+1:]
        if len(code) == 8:
            report_values.append([code, child.text])

    parsed_df = pd.DataFrame(report_values, columns = ['MDRM_Item','Value'])
    MDRM_df = wsio.ReadCSV(paths.folder_Orig + paths.filename_MDRM)
    full_df=parsed_df.set_index('MDRM_Item').join(MDRM_df.set_index('MDRM_Item'))
    #print(len(tot[pd.isnull(tot.Item_Name)]))
    full_df.reset_index(level=0, inplace=True)
    return full_df


tree  = ET.parse(filepath)
root = tree.getroot()

report_values = []
for child in root:
    #print(child.tag, child.attrib)
    code = child.tag[child.tag.find('}')+1:]
    if len(code) == 8:
        report_values.append([code, child.text])

parsed_df = pd.DataFrame(report_values, columns = ['MDRM_Item','Value'])
MDRM_df = wsio.ReadCSV(paths.folder_Orig + paths.filename_MDRM)
full_df=parsed_df.set_index('MDRM_Item').join(MDRM_df.set_index('MDRM_Item'))
#print(len(tot[pd.isnull(tot.Item_Name)]))
full_df.reset_index(level=0, inplace=True)






In [ ]:
pres_file_path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-pres.xml'
cap_file_path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'

path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-pres.xml'
pres_search_string = '/*pres.xml'

In [ ]:
def remove_text(input_data):
    texts_to_remove = [
        '{http://www.w3.org/1999/xlink}',
        '{http://www.xbrl.org/2003/linkbase}'
        ]
    
    def removeText(text, texts_to_remove):
        for string in texts_to_remove:
            text = text.replace(string, "")
        return text

    if isinstance(input_data, str):
        return removeText(input_data, texts_to_remove)
    elif isinstance(input_data, dict):
        return {removeText(key, texts_to_remove) if isinstance(key, str) else key: removeText(value, texts_to_remove) if isinstance(value, str) else value
                for key, value in input_data.items()}
    else:
        raise TypeError("Input data must be either a str or a dict")


In [ ]:
import xml.etree.ElementTree as ET
import FDIC.constants as paths
import pandas as pd
import glob
import regex as re


def safe_get_prefix(attribute, key, prefix_length=3):
    value = attribute.get(key)
    return value[:prefix_length] if value else None


def get_ReportAndDate(filename) -> tuple[str, str]:

    #File is formatted as follows:  "call-report051-2023-12-31-v265-pres.xml"
    #search_pattern = '([^/]+$)'
    search_pattern = 'Form[0-9]{3}'
    fn_report = re.search(search_pattern,filename).group()
    #fn_date_split = filename.replace('call-report','').split('-')
    fn_date_split = filename.split('-')
    fn_date = f'{fn_date_split[2]}-{fn_date_split[3]}-{fn_date_split[4]}' if (fn_date_split[2] and fn_date_split[3] and fn_date_split[4]) else None
    return (fn_report, fn_date)


# Print the structure of the presentation file
# These taxonomies have a very particular and nuanced structure
def parse_pres(input_path= paths.localPath + paths.folder_Taxonomies, input_search_pattern= '*/*-pres.xml', input_df= None, output_filepath= paths.folder_Orig + paths.filename_MDRM):

    pres_dtype = {'from':str, 'to':str, 'date':str, 'report':str}
    pres_df = pd.DataFrame(columns=['from', 'to', 'date', 'report']) #, dtype=pres_dtype)

    for file_path in glob.glob(input_path + input_search_pattern):

        file_df = pd.DataFrame(columns=['from', 'to', 'date', 'report']) #, dtype=pres_dtype)

        # Parse the presentation linkbase file
        xml_tree = ET.parse(file_path)
        xml_root = xml_tree.getroot()

        for child in xml_root:
            #print(child.tag, child.attrib)

            if child.tag[child.tag.find('}')+1:] == 'presentationLink':
                for i,ch in enumerate(child):

                    tag = remove_text(ch.tag)
                    attrib = remove_text(ch.attrib)
                    label_prefix = safe_get_prefix(attrib, 'label')
                    from_prefix = safe_get_prefix(attrib, 'from')
                    to_prefix = safe_get_prefix(attrib, 'to')

                    if tag == 'presentationArc' and (label_prefix == 'cc_' or to_prefix == 'cc_'):
                        #att_label = attrib.get("label", "N/A")
                        att_from = attrib.get('from', 'N/A')
                        att_to = attrib.get("to", "N/A")
                        if att_from:
                            att_report, att_date = get_ReportAndDate(file_path) 
                            file_data = pd.DataFrame({'from': att_from, 'to': att_to, 'date':att_date, 'report':att_report}, index=[0]) #, dtype=pres_dtype)
                            file_df = pd.concat([file_df, file_data])
        
        pres_df = pd.concat([pres_df, file_df], ignore_index=True)
    pres_df.to_csv(output_filepath, sep=',', quotechar='"', index= False)  



def parse_cap(input_path= paths.localPath+paths.folder_Taxonomies, input_search_pattern = '*/*-cap.xml', input_df= None, output_filepath= paths.folder_Orig+paths.filename_MDRM):
    cap_dtype = {'from':str, 'to':str, 'label':str, 'text':str}
    cap_df = pd.DataFrame(columns=['from', 'to', 'label', 'text']) #, dtype=cap_dtype)

    for file_path in glob.glob(input_path + input_search_pattern):

        file_df = pd.DataFrame(columns=['from', 'to', 'label', 'text']) #, dtype=cap_dtype)

        # Parse the presentation linkbase file
        xml_tree = ET.parse(file_path)
        xml_root = xml_tree.getroot()

        for child in xml_root:
            #print(child.tag, child.attrib)

            if child.tag[child.tag.find('}')+1:] == 'labelLink':
                for i,ch in enumerate(child):

                    tag = remove_text(ch.tag)
                    attrib = remove_text(ch.attrib)

                    if tag == 'labelArc':
                        att_from = attrib.get('from', 'N/A')
                        att_to = attrib.get('to', 'N/A')
                        
                    if tag == 'label':
                        att_label = attrib.get('label', 'N/A')
                        att_text = ch.text
                        
                        if att_label:
                            file_data = pd.DataFrame({'from': att_from, 'to': att_to, 'label':att_label, 'text':att_text}, index=[0]) #, dtype=pres_dtype)
                            file_df = pd.concat([file_df, file_data])
                            #Reset values of all attribute variables
                            att_from, att_to, att_label, att_text = None, None, None, None


                    #[2 iterations to fill each of these]
                    # check row label = labelArc
                        #set values
                    # check row label = label
                        #set values

                    #[Run every 2 iterations or when all vals are full??]
                    # if labelArc or label
                        #set df values; append to df

        cap_df = pd.concat([cap_df, file_df], ignore_index=True)
    cap_df.to_csv(output_filepath, sep=',', quotechar='"', index= False)  



def merge_dataframes(dataframes_and_mergeColumns: list[tuple[pd.DataFrame, str]], is_filepath=False, how='inner'):
    """
    Merges multiple dataframes into a single dataframe based on specified columns.

    :param dataframes_and_mergeColumns: List of tuples containing either DataFrames or file paths and their corresponding
                                merge column names. 
                                Format: [(df_or_path, merge_column), (df_or_path, merge_column), ...]
    :param filepaths: Boolean indicating if the input is a list of file paths. Default is False.
    :param how: Type of merge to be performed. Default is 'inner'. Other options: 'outer', 'left', 'right'.
    :return: Merged pandas DataFrame.
    """
    if is_filepath:
        # Load DataFrames from CSV files
        dataframes = [(pd.read_csv(df_or_path), col) for df_or_path, col in dataframes_and_mergeColumns]
    else:
        dataframes = dataframes_and_mergeColumns

    # Initialize merged_df with the first DataFrame and its corresponding merge column
    merged_df, left_on = dataframes[0]

    for df, right_on in dataframes[1:]:
        # Merge on specified columns from each DataFrame
        merged_df = pd.merge(merged_df, df, left_on=left_on, right_on=right_on, how=how)
        left_on = right_on  # Update left_on for the next merge


    return merged_df

# Example usage:
# If you have a list of file paths and their corresponding columns:
# merged_df = merge_dataframes([("file1.csv", "col1"), ("file2.csv", "col2"), ("file3.csv", "col3")],
#                              filepaths=True)

# If you have a list of DataFrames and their corresponding columns:
# merged_df = merge_dataframes([(df1, "col1"), (df2, "col2"), (df3, "col3")],
#                              filepaths=False)





In [ ]:
#cap TEST
#path = paths.localPath + paths.folder_Taxonomies
#file_search_pattern = '*/*-cap.xml'
file_search_pattern = ''
path = '/Users/kris/data_sources/fdic/Src/Taxonomies/Call_ 12312023_Form051/call-report051-2023-12-31-v265-cap.xml'
output = 'FDIC/MDRM_cap_test.csv'

parse_cap(input_path=path, input_search_pattern=file_search_pattern, output_filepath=output)


In [ ]:
pres_file = 'FDIC/MDRM_test.csv'
cap_file = 'FDIC/MDRM_cap_test.csv'

pres_df = pd.read_csv(pres_file, delimiter=',')
cap_file = pd.read_csv(cap_file, delimiter=',')

#test = merge_dataframes([pres_df, cap_file], ['from', 'from'])
test = merge_dataframes([(pres_df, 'from'), (cap_file, 'from')])


In [55]:
def build_MDRMdict(input_path =paths.localPath+paths.folder_MDRMs+paths.filename_MDRM_src, export_path =paths.folder_Orig + paths.filename_MDRM, filters=None):
    """
    Reads a CSV file, filters the DataFrame based on the given criteria, 
    and exports the result to a new CSV file.

    Parameters:
    - input_path: str, path to the input CSV file.
    - export_path: str, path where the filtered DataFrame should be saved.
    - filters: dict, optional. A dictionary where keys are column names and values 
               are lists of filter criteria.

    Example:
    build_MDRMdict(
        input_path='path/to/MDRM_CSV.csv',
        export_path='path/to/export/MDRM_dict.csv',
        filters={'Reporting Form': ['FFIEC 031', 'FFIEC 041', 'FFIEC 051']})
    """

    def format_str_date(date_string):
        try:
            date_part = date_string.split()[0]
            month, day, year = date_part.split('/')
            return f"{year.zfill(4)}-{month.zfill(2)}-{day.zfill(2)}"
        except:
            return date_string

    # Read the CSV file with headers starting on line 2
    df = pd.read_csv(input_path, header=1, sep=',', quotechar='"') #, parse_dates=['Start Date', 'End Date'])
    df.dropna(axis=1, how='all', inplace=True)

    df[['Start Date', 'End Date']] = df.apply(lambda row: pd.Series([
        format_str_date(row['Start Date']),
        format_str_date(row['End Date'])]),
        axis=1)

    # Apply filters if provided
    if filters:
        for column, values in filters.items():
            df = df[df[column].isin(values)]
    
    #Rename columns and format df to needed format and order
    rename_columns={'Start Date':'Start_Date',
                    'End Date':'End_Date',
                    'Item Name':'Item_Name',
                    'Confidentiality':'Confidential',
                    'ItemType':'Item Type',
                    'Reporting Form':'Reporting_Form',
                    'SeriesGlossary':'Series Glossary'}
    df.rename(columns=rename_columns, inplace=True)
    df['MDRM_Item'] = df['Mnemonic'] + df['Item Code'].astype(str)
    df = df[['MDRM_Item', 'Start_Date', 'End_Date', 'Item_Name', 'Confidential', 'Reporting_Form']]

    # Export the filtered DataFrame to the specified export path
    if export_path:
        df.to_csv(export_path, index=False)
    else:
        return df


In [ ]:
#mdf[mdf['End_Date'] > '1999-12-31']
#mdf=mdf.drop(columns='Reporting_Form').drop_duplicates()
mdf


In [ ]:
#filters = {'Reporting Form':['FFIEC 031', 'FFIEC 041', 'FFIEC 051']}
filters = {'Mnemonic':['CALL',
                            'CENB','IADX','IBFQ','RCCD','RCCF','RCEG',
                       'RCFD',
                            'RCFA', 'RCFW',
                       'RCFN',
                       'RCON',
                          'RCF0','RCF1','RCF2','RCF3','RCF4','RCF5','RCF6','RCF7','RCF8','RCF9','RCOA','RCOW',
                       'RCOS','RIAD','RIAS','RIDM','RIFN','SCHJ',
                  ]}
#filters = {'Mnemonic':['CALL']}


#build_MDRMdict(export_path =paths.folder_Orig + 'MDRM_test.csv' ,filters=filters)
#build_MDRMdict(export_path =paths.folder_Orig + 'MDRM_Dict.csv', filters=filters)
mdf =build_MDRMdict(export_path =None, filters=filters)
mdf =mdf[mdf['End_Date'] > '1999-12-31']
mdf

In [57]:
def ParseXBRL(filepath):
    """
    Parses Call Reports to extract all MDRM item and value from XBRL file
    """

    def clean_xbrl_tag(text):
        remove_text = [
                        '{http://www.w3.org/1999/xlink}',
                        '{http://www.xbrl.org/2003/linkbase}',
                        '{http://www.xbrl.org/2003/instance}',
                        '{http://www.ffiec.gov/xbrl/call/concepts}'
                    ]
           
        for rt in remove_text:
            text = text.replace(rt, "")
        return text

    xlink = '{http://www.w3.org/1999/xlink}'

    # Parse XML
    tree  = ET.parse(filepath)
    root = tree.getroot()
    reporting_form = None

    report_values = []
    for child in root:
        #print(child.tag, child.attrib)
        #code = child.tag[child.tag.find('}')+1:]
        code = clean_xbrl_tag(child.tag)

        if code == 'schemaRef' and reporting_form is None:
            #get attrib=href
            #parse url for regex='report{0-9}*3'
            attrib = child.attrib.get(xlink+'href')
            if attrib:
                reporting_form = re.search(pattern=r'report\d{3}', string=attrib)
                reporting_form_str = reporting_form.group(0) if reporting_form else None

        data_unit = child.attrib.get('unitRef')
        data_decimals = child.attrib.get('decimals')
        
        #Infer datatype based on basic logic and value
        if data_decimals:
            if int(data_decimals) == 0:
                data_type = 'int'
            elif int(data_decimals) >0:
                data_type = 'float'
            else:
                raise TypeError()
        elif (child.text and child.text.strip().lower() in ['true','false']):
            data_type = 'bool'
        else:
            data_type = 'str'

        #NEED SOMETHING MORE ROBUST TO CAPTURE VALUES HERE
        if len(code) == 8:
            report_values.append([code, child.text, data_type, data_unit, data_decimals, reporting_form_str])

    parsed_df = pd.DataFrame(report_values, columns = ['MDRM_Item','Value','Datatype','Unit','Decimals', 'Reporting_Form_CallReport'])
    MDRM_df = wsio.ReadCSV(paths.folder_Orig + paths.filename_MDRM)
    full_df=parsed_df.set_index('MDRM_Item').join(MDRM_df.set_index('MDRM_Item'))
    #print(len(tot[pd.isnull(tot.Item_Name)]))
    full_df.reset_index(level=0, inplace=True)
    return full_df

In [84]:
#WIP - DATATYPE 
def build_MDRM_datatype(filepath_MDRM, folderpath_callReport, folderpath_searchString='/*.XBRL', filepath_MDRM_out=paths.folder_Orig+paths.filename_MDRM):

    MDRM_df = pd.read_csv(filepath_MDRM, sep=',', quotechar='"', header=0)
    MDRM_df.drop('Reporting Form', axis=1, errors='ignore')

    bank_df = pd.DataFrame()    
    if folderpath_callReport[-1:] == '/':
        folderpath_callReport = folderpath_callReport[:-1]

    bank_foldername = folderpath_callReport.rsplit('/',1)[-1]
    rssd = bank_foldername[bank_foldername.rfind('_')+1:]


    for call_report_filename in glob.glob(folderpath_callReport + folderpath_searchString):
        period_date = call_report_filename[call_report_filename.rfind('_')+1:len(call_report_filename)-5]
        period_date = datetime.datetime.strptime((call_report_filename[call_report_filename.rfind('_')+1:len(call_report_filename)-5]), '%Y%m%d').date()
        #print(period_date, bank_name, rssd)
        tb_df = ParseXBRL(call_report_filename)
        #print(tdf)
        tb_df['ReportPeriodEndDate'] =period_date
        tb_df['RSSD_ID'] =rssd
        tb_df = tb_df[['MDRM_Item','Datatype','Unit','Decimals', 'ReportPeriodEndDate']] #, 'Reporting_Form']]
        try:
            #bank_df = pd.concat([bank_df,tb_df])
            bank_df = pd.concat([bank_df,tb_df], ignore_index=True).drop_duplicates(['MDRM_Item'], keep='first') #,'Reporting_Form'], keep='first')
        except NameError:
            bank_df = tb_df
    
    #Set types of merge item
    MDRM_df = MDRM_df.astype(dtype={'MDRM_Item':'string'})
    bank_df = bank_df.astype(dtype={'MDRM_Item':'string'})
    
    #Merge dataframes
    merged_MDRM_df = pd.merge(MDRM_df, bank_df, how='right', on='MDRM_Item', left_index=False, right_index=False, suffixes=['_l', '_r']) 

    if filepath_MDRM_out:
        wsio.WriteDataFrame(filepath_MDRM_out, merged_MDRM_df)
    else:
        return merged_MDRM_df

In [ ]:
filepath_MDRM = '/Users/kris/Library/CloudStorage/OneDrive-Personal/Documents/Visual Studio 2019/Repos/ffiec-xbrl/PubData/MDRM_Dict.csv'
folderpath_bank = '/Users/kris/data_sources/fdic/BulkReports/1ST BANK OF SEA ISLE CITY_148470/'

filepath_MDRM_out = '/Users/kris/Library/CloudStorage/OneDrive-Personal/Documents/Visual Studio 2019/Repos/ffiec-xbrl/PubData/MDRM_Dict_2.csv'

mdf = build_MDRM_datatype(filepath_MDRM, folderpath_bank, filepath_MDRM_out=None)
mdf

In [ ]:
mdf


In [ ]:
#xdf =mdf.join(fdf, on='MDRM_Item', how='outer', lsuffix='_l', rsuffix='_r')
xdf =pd.merge(mdf, fdf, how='outer', on='MDRM_Item', left_index=False, right_index=False, suffixes=['_l', '_r'])
xdf

In [ ]:
filepath = '/Users/kris/data_sources/fdic/BulkReports/1ST BANK OF SEA ISLE CITY_148470/1ST BANK OF SEA ISLE CITY_148470_20240331.XBRL'

df = ParseXBRL(filepath)
#df





In [ ]:
#df[df['Datatype']=='bool']
#df[df['Value']==None]
df

In [19]:
import re

attrib = 'http://www.ffiec.gov/xbrl/call/report05/2018-03-31/v161/call-report05-2018-03-31-v161.xsd'

report = re.search(pattern=r'report\d{3}', string=attrib)
report_str = report.group(0) if report else None

In [ ]:
folders = glob.glob(paths.localPath + paths.folder_BulkReports + '*')
folders

In [ ]:
def format_str_date(date_string):
    try:
        date_part = date_string.split()[0]
        month, day, year = date_part.split('/')
        return f"{year.zfill(4)}-{month.zfill(2)}-{day.zfill(2)}"
    except:
        return date_string
            
df = pd.read_csv(input_path, header=1, sep=',', quotechar='"') #, parse_dates=['Start Date', 'End Date'])
df.dropna(axis=1, how='all', inplace=True)

df[['Start Date', 'End Date']] = df.apply(lambda row: pd.Series([
    format_str_date(row['Start Date']),
    format_str_date(row['End Date'])]),
    axis=1)

# Apply filters if provided
if filters:
    for column, values in filters.items():
        df = df[df[column].isin(values)]

#Rename columns and format df to needed format and order
rename_columns={'Start Date':'Start_Date',
                'End Date':'End_Date',
                'Item Name':'Item_Name',
                'Confidentiality':'Confidential',
                'ItemType':'Item Type',
                'Reporting Form':'MDRMReport_Form',
                'SeriesGlossary':'Series Glossary'}
df.rename(columns=rename_columns, inplace=True)
df['MDRM_Item'] = (df['Mnemonic'] + df['Item Code'].astype(str)).astype(str)
#df = df.astype(dtype={'MDRM_Item':'string'})
df = df[['MDRM_Item', 'Start_Date', 'End_Date', 'Item_Name', 'Confidential']]

In [ ]:
#BUILD RSSD_DICT FROM POR - #Clean RSSD_Por Format - Align with and Create RSSD_Dict.csv
def Gen_RSSDict_from_PORfiles(folderpath_input=paths.localPath + paths.folder_RSSDs, filepath_output=paths.folder_Orig + paths.filename_RSSD): 
    
    dtypes = {'IDRSSD': np.dtype('str'),
          'FDIC Certificate Number': np.dtype('str'),
          'OCC Charter Number': np.dtype('str'),
          'OTS Docket Number': np.dtype('str'),
          'Primary ABA Routing Number': np.dtype('str'),
          'Financial Institution Name': np.dtype('str'),
          'Financial Institution Address': np.dtype('str'),
          'Financial Institution City': np.dtype('str'),
          'Financial Institution State': np.dtype('str'),
          'Financial Institution Zip Code': np.dtype('str'),
          'Financial Institution Filing Type': np.dtype('str'),
          'Last Date/Time Submission Updated On': np.dtype('str')}
    
    f_df = pd.DataFrame()
    
    #Open File (Bank_dim from RSSD_Por)
    for file in glob.glob(folderpath_input + '* POR *.txt'):
        t_df = pd.read_csv(file, sep='\t', index_col=False, quotechar='"', dtype = dtypes) #, parse_dates=['Last Date/Time Submission Updated On'])


        #Clean RSSD_Por File
        t_df = t_df.apply(lambda x: x.str.strip() if x.dtype.name == 'object' else x, axis=0)

        t_df.rename(columns={'IDRSSD': 'RSSD_ID'}, inplace=True)
        t_df.columns = t_df.columns.str.replace(' ', '_')

        f_df = pd.concat([t_df, f_df])

    if filepath_output:
        #Create RSSD_Dict file
        f_df.to_csv(filepath_output, sep=',', quotechar='"', index= False)
    else:
        return f_df


Gen_RSSDict_from_PORfiles(filepath_output=None)

In [ ]:




path = '/Users/kris/data_sources/fdic/BulkReports/1ST BANK OF SEA ISLE CITY_148470'


In [ ]:
import glob
import os

base_path = '/Users/kris/data_sources/fdic/BulkReports/'
folder_path = '/Users/kris/data_sources/fdic/BulkReports/1ST BANK OF SEA ISLE CITY_148470/'
folder_paths = [folder_path]
file_path = '/Users/kris/data_sources/fdic/BulkReports/1ST BANK OF SEA ISLE CITY_148470/1ST BANK OF SEA ISLE CITY_148470_20240331.XBRL'

print(os.path.abspath(folder_path))

In [ ]:
path = base_path

folders = [folder for folder in glob.glob(path + '*') if not(os.path.isfile(folder))]
print(folders)

In [ ]:
folder_paths = []
f = folder_paths or folders
f

In [6]:
import pandas as pd
import FDIC.constants as paths

MDRM_df = pd.read_csv(paths.folder_Orig + paths.filename_MDRM)



In [7]:
MDRM_df
MDRM_df[MDRM_df['MDRM_Item']=='RCOA3792']

,MDRM_Item,Start_Date,End_Date,Item_Name,Confidential,Reporting_Form_Num,Datatype,Unit,Decimals
1089,RCOA3792,2014-03-31,9999-12-31,TOTAL QUALIFYING CAPITAL ALLOWABLE UNDER THE R...,N,41,int,USD,0.0
2834,RCOA3792,2017-03-31,9999-12-31,TOTAL QUALIFYING CAPITAL ALLOWABLE UNDER THE R...,N,51,int,USD,0.0
